In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
%%capture
!pip install transformers sentencepiece boto3 sacrebleu datasets --quiet

In [ ]:
%%capture
!pip install nltk -U --quiet

In [ ]:
%%capture
!pip install jsonlines

In [ ]:
from datasets import load_dataset, load_metric
import os
import boto3
import torch
import shutil
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, MarianMTModel, MarianTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import MBart50TokenizerFast, MBartForConditionalGeneration
from tqdm.notebook import tqdm
import logging
from torch.utils.data import DataLoader
from datetime import datetime
from sklearn.model_selection import train_test_split
from collections import defaultdict
import jsonlines
import json
from jsonlines import jsonlines
import numpy as np

tqdm.pandas()
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# cnc_path = '/content/drive/MyDrive/CausalNewsCorpus/data' 
# def process_cnc_data(data_type):
#   df = pd.read_csv(os.path.join(cnc_path, f"{data_type}.csv"))
#   df.drop(['corpus', 'doc_id', 'sent_id', 'eg_id', 'index', 'seq_label', 'pair_label', 'context', 'num_sents'], axis=1, inplace=True)
#   df.rename(columns={
#       'text' : 'input_text',
#       'text_w_pairs': 'target_text'
#   }, inplace=True)
#   df.to_csv(f'/content/{data_type}.csv', index=False)
# process_cnc_data('train_subtask2')
# process_cnc_data('dev_subtask2')

In [ ]:
train_model_checkpoints = "facebook/mbart-large-50"
# train_model_checkpoints = "t5-large"

In [ ]:
# load the MarianMT tokenizer
tokenizer = MBart50TokenizerFast.from_pretrained(train_model_checkpoints, src_lang="en_XX", tgt_lang="en_XX")
# tokenizer = T5Tokenizer.from_pretrained(train_model_checkpoints)

In [ ]:
# def add_prompt(source_list, target_list):
#   processed_input = []
#   for input, target in zip(source_list, target_list):
#     ts_ratio = len(target)/len(input)
#     if ts_ratio < 0.95:
#       prefix = "paraphrase short"
#     elif ts_ratio >= 0.95 and ts_ratio <= 1.10:
#       prefix = "paraphrase normal"
#     else:
#       prefix = "paraphrase long"
#     input = prefix + " " + input
#     processed_input.append(input)
#   return processed_input

In [ ]:
# session = boto3.Session(
#     aws_access_key_id='**********************',
#     aws_secret_access_key='****************************',
# )
# s3 = session.resource('s3')
# key = "unicausal_data"
# filename = "./unicausal.csv"
# print(key)
# s3.meta.client.download_file(Bucket='tsd2022', Key=key, Filename=filename)

In [ ]:
session = boto3.Session(
    aws_access_key_id='***********************',
    aws_secret_access_key='**********************',
)
s3 = session.resource('s3')
key = "CASE2022-train-subtask2"
filename = "./train_subtask2.csv"
print(key)
s3.meta.client.download_file(Bucket='tsd2022', Key=key, Filename=filename)

In [ ]:
# preprocess MUST-C dataset
max_input_length = 128 
max_target_length = 128
def preprocess_function(examples):
    inputs = examples["input_text"]
    targets = examples["target_text"]
    inputs = [f"CASE: {text}" for text in inputs]
   # inputs = add_prompt(inputs, targets) # append appropriate prompts 
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
current_directory = os.getcwd()
path_to_train_data = os.path.join(current_directory, "./train_subtask2.csv")
# path_to_validation_data = os.path.join(current_directory, "dev_subtask2.csv")
raw_train_dataset = load_dataset('csv', data_files={"train": path_to_train_data})
# raw_validation_dataset = load_dataset('csv', data_files={"validation": path_to_validation_data})

In [ ]:
raw_train_dataset

In [ ]:
# tokenize raw data
tokenized_train_datasets = raw_train_dataset["train"].map(preprocess_function, batched=True)
# tokenized_validation_datasets = raw_validation_dataset["validation"].map(preprocess_function, batched=True)

In [ ]:
# session = boto3.Session(
#     aws_access_key_id='************************',
#     aws_secret_access_key='*********************',
# )
# s3 = session.resource('s3')
# key = "CASE22_fewshot-learning-mbart-large-50-finetuned-for-span-detection"
# filename = "fewshot-learning-mbart-large-50-finetuned-for-span-detection.zip"
# print(key)
# s3.meta.client.download_file(Bucket='tsd2022', Key=key, Filename=filename)

In [ ]:
!unzip ./fewshot-learning-mbart-large-50-finetuned-for-span-detection.zip -d ./fewshot-learning-mbart-large-50-finetuned-for-span-detection-cluster

In [ ]:
# training procedure
model_checkpoints = "./fewshot-learning-mbart-large-50-finetuned-for-span-detection-cluster/checkpoint-7300/"
model = MBartForConditionalGeneration.from_pretrained(model_checkpoints)
# model = T5ForConditionalGeneration.from_pretrained(train_model_checkpoints, return_dict=True)

In [ ]:
batch_size = 4 # change batch-size according to GPU availability 
model_name = train_model_checkpoints.split("/")[-1]
epoch = 20
strategy = "epoch"

# define training model arguments
args = Seq2SeqTrainingArguments(
    f"fewshot-learning-{model_name}-finetuned-for-span-detection",
    learning_rate=5e-5, 
    logging_strategy=strategy,
    # learning_rate=0.0003,
    # lr_scheduler_type="linear",
    # warmup_ratio=0.06,
    # optim="adafactor",
    save_strategy=strategy,
    # save_steps=save_steps_,
    # evaluation_strategy=strategy,
    # eval_steps=steps_,
    per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    # report_to="wandb",
    save_total_limit=1,
    predict_with_generate=True    
)

# initialize data-collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
sacrebleu = load_metric("sacrebleu")
meteor = load_metric("meteor")
f1_metric = load_metric("f1")

# import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
    
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    f1_result = f1_metric.compute(predictions=decoded_preds, references=decoded_labels)
    sacrebleu_result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    result = {
        "bleu": sacrebleu_result["score"],
        "meteor": meteor_result['meteor'],
        "f1": f1_result["f1"],
    }
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    print(result)
    return result

In [ ]:
# initialize the trainer module
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_datasets,
    # eval_dataset=tokenized_validation_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# train the model
trainer.train()

In [ ]:
devices = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# model_checkpoints = "./drive/MyDrive/CASE22/fewshot-learning-mbart-large-50-finetuned-for-span-detection/checkpoint-460"
model_checkpoints = "./fewshot-learning-mbart-large-50-finetuned-for-span-detection/checkpoint-5834"
# model_checkpoints = "./drive/MyDrive/CASE22/fewshot-learning-t5-large-finetuned-for-span-detection/checkpoint-920"
tokenizer = MBart50TokenizerFast.from_pretrained(train_model_checkpoints, src_lang="en_XX", tgt_lang="en_XX")
model = MBartForConditionalGeneration.from_pretrained(model_checkpoints).to(devices)
# tokenizer = T5Tokenizer.from_pretrai  ned(model_checkpoints)
# model = T5ForConditionalGeneration.from_pretrained(model_checkpoints, return_dict=True).to(devices)

In [ ]:
pred_list = []
df = pd.read_csv('./dev_subtask2.csv')
for _, row in tqdm(df.iterrows(),total=df.shape[0]):
  decoded_output = model.generate(
      **tokenizer(f"CASE: {row['input_text']}", return_tensors='pt').to(devices),
      max_length=256
  )
  output = [tokenizer.decode(e,skip_special_tokens=True) for e in decoded_output][0]
  pred_list.append(output)

df['pred'] = pred_list
df.to_csv(f"./proc_dev_subtask2_{model_name}.csv", index=False)

# !mv ./fewshot-learning-t5-large-finetuned-for-span-detection/ ./drive/MyDrive/CASE22/fewshot-learning-t5-large-finetuned-for-span-detection/
# !mv ./fewshot-learning-mbart-large-50-finetuned-for-span-detection/ ./drive/MyDrive/CASE22/fewshot-learning-mbart-large-50-finetuned-for-span-detection

In [ ]:
df

In [ ]:
df.iloc[16]['pred'], df.iloc[17]['pred']

In [ ]:
sample = df.iloc[16]
print(sample['input_text'])

# decoded_output = model.generate(
#     **tokenizer(sample['input_text'], return_tensors='pt').to(devices)
# )
decoded_output = model.generate(
    **tokenizer(f"CASE: {sample['input_text']}", return_tensors='pt').to(devices), 
    # do_sample=True,
    # top_p=0.95, 
    # top_k=50,
    num_return_sequences=5
)
# decoded_output = model.generate(
#     **tokenizer(sample['input_text'], return_tensors='pt').to(devices), 
#     num_beams=10,
#     early_stopping=True, 
#     num_return_sequences=4
# )

In [ ]:
output = [tokenizer.decode(e,skip_special_tokens=True) for e in decoded_output]

In [ ]:
output

In [ ]:
# current_directory = os.getcwd()
# model_checkpoints = f"fewshot-learning-{model_name}-finetuned-for-span-detection"
# model_checkpoint_directory = os.path.join(current_directory, model_checkpoints)
# print(model_checkpoint_directory)   
# shutil.make_archive(model_checkpoint_directory, "zip", model_checkpoint_directory.split('/')[-1])

In [ ]:
# session = boto3.Session(
#     aws_access_key_id='******************',
#     aws_secret_access_key='*************************',
# )
# s3 = session.resource('s3')
# key = f"COLAB_CASE22_{model_checkpoints}"
# filename = os.path.join(current_directory, f"{model_checkpoints}.zip")
# print(key)
# s3.meta.client.upload_file(Bucket='tsd2022', Key=key, Filename=filename)

In [ ]:
pred_list = []
df = pd.read_csv('./dev_subtask2_text.csv')
# df.drop_duplicates(subset=['input_text'], inplace=True)
# df.reset_index(drop=True, inplace=True)

output_list = [] 

for index, row in tqdm(df.iterrows(),total=df.shape[0]):
  current_dict = dict()
  current_dict['index'] = index
  decoded_output = model.generate(
      **tokenizer(f"CASE: {row['text']}", return_tensors='pt').to(devices),
      num_return_sequences=3
  )
  output = [tokenizer.decode(e,skip_special_tokens=True) for e in decoded_output]
  current_dict['prediction'] = output
  # current_dict['target_text'] = row['target_text']
  output_list.append(current_dict)

In [ ]:
with jsonlines.open('devsubtask2_submission_DF.json','w') as writer:
  writer.write_all(output_list)

In [ ]:
pred_list = []
df = pd.read_csv('./test_subtask2_text.csv')
# df.drop_duplicates(subset=['input_text'], inplace=True)
# df.reset_index(drop=True, inplace=True)

output_list = [] 

for index, row in tqdm(df.iterrows(),total=df.shape[0]):
  current_dict = dict()
  current_dict['index'] = index
  decoded_output = model.generate(
      **tokenizer(f"CASE: {row['text']}", return_tensors='pt').to(devices),
      num_return_sequences=3
  )
  output = [tokenizer.decode(e,skip_special_tokens=True) for e in decoded_output]
  current_dict['prediction'] = output
  # current_dict['target_text'] = row['target_text']
  output_list.append(current_dict)

In [ ]:
with jsonlines.open('testsubtask2_submission_DF.json','w') as writer:
  writer.write_all(output_list)